In [67]:
import openai
import pandas as pd
import dotenv
import os

dotenv.load_dotenv()

True

In [68]:
client = openai.Client(api_key=os.getenv("openaiKey"))

In [69]:
file = client.files.create(
    file=open("sales_data.csv","rb"),
    purpose="assistants"
)

In [70]:
print(file.id)

file-UWwqyovKyZmgU1s7vQSG2Z


In [71]:
assistant = client.beta.assistants.create(
    name="analista de dados",
    instructions="você e um analista que analisa dados sobre venda",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)

In [72]:
pergunta = "gere um grafico de pizza com o percentual de vendas por linha de produto"

In [73]:
thread = client.beta.threads.create()
mensagens = client.beta.threads.messages.create(thread_id=thread.id,role="user",content=pergunta)

In [74]:
run = client.beta.threads.runs.create(thread_id=thread.id,assistant_id=assistant.id,instructions="nome de usuario premium")


In [75]:
import time
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)


In [76]:
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(thread_id=thread.id)
    print(mensagens)
else:
    print(f"erro{run.status}")

SyncCursorPage[Message](data=[Message(id='msg_XoJPZ4nYzTnzX3x5UY5qiI17', assistant_id='asst_4qF4LM7CYSIS7nRTCEvwwGex', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-GxFntVerM9JBE7iqdmBJ3f', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza mostrando o percentual de vendas por linha de produto. Cada fatia representa a proporção de vendas de cada categoria em relação ao total. Se precisar de mais alguma coisa ou de algum detalhe específico, sinta-se à vontade para perguntar!'), type='text')], created_at=1740590712, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_aLCEWN5ca4QuZ9cQMjzU5YjV', status=None, thread_id='thread_9Xegj3kGcFtoUk4WwYG3I6Wv'), Message(id='msg_6EsATdtl4hh1BlBrHiqSLq6N', assistant_id='asst_4qF4LM7CYSIS7nRTCEvwwGex', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(anno

In [77]:
run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id,run_id=run.id)


In [78]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")


====Step message_creation
Para gerar um gráfico de pizza com o percentual de vendas por linha de produto, primeiro preciso examinar o conteúdo do arquivo que você enviou. Vamos começar carregando e analisando os dados do arquivo.

====Step tool_calls
import pandas as pd

# Carregar o arquivo enviado para um DataFrame do Pandas
file_path = '/mnt/data/file-UWwqyovKyZmgU1s7vQSG2Z'
data = pd.read_csv(file_path)

# Exibir as primeiras linhas do DataFrame para entender sua estrutura
data.head()

====Step message_creation
O arquivo contém colunas relacionadas a vendas, como "Product line" e "Total". Vamos calcular o total de vendas para cada linha de produto e, em seguida, usar essas informações para gerar um gráfico de pizza mostrando o percentual de vendas por linha de produto.

====Step tool_calls
import matplotlib.pyplot as plt

# Calcular o total de vendas por linha de produto
sales_by_product_line = data.groupby('Product line')['Total'].sum()

# Calcular o percentual de vendas por linh